In [5]:
!pip install openai groq --quiet

import os
from openai import OpenAI


In [6]:
os.environ["GROQ_API_KEY"]='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'

In [22]:
client = OpenAI(api_key=os.environ["GROQ_API_KEY"], base_url='https://api.groq.com/openai/v1')

In [19]:
class ConversationManager:
    def __init__(self, model="llama-3.1-8b-instant", k=3):
        self.history = []  # stores dicts {"role": "user/assistant/system", "content": "..."}
        self.summaries = []  # stores periodic summaries
        self.model = model
        self.k = k  # summarize every k turns
        self.run_count = 0

    def add_message(self, role, content):
        """Add a message to the conversation history"""
        self.history.append({"role": role, "content": content})
        self.run_count += 1

        # Check if periodic summarization needed
        if self.run_count % self.k == 0:
            self.summarize_history()

    def summarize_history(self):
        """Summarize conversation history and replace with summary"""
        prompt = [
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations concisely."},
            {"role": "user", "content": f"Summarize this conversation:\n\n{self.history}"}
        ]

        response = client.chat.completions.create(
            model=self.model,
            messages=prompt,
            max_tokens=150
        )

        summary = response.choices[0].message.content.strip()
        self.summaries.append(summary)

        self.history = [{"role": "system", "content": f"Summary so far: {summary}"}]

    def truncate_by_turns(self, n=5):
        """Keep only the last n messages"""
        if len(self.history) > n:
            self.history = self.history[-n:]

    def truncate_by_length(self, max_chars=500):
        """Keep only up to max_chars of recent conversation"""
        total = 0
        truncated = []
        for msg in reversed(self.history):
            if total + len(msg["content"]) <= max_chars:
                truncated.insert(0, msg)
                total += len(msg["content"])
            else:
                break
        self.history = truncated

    def show_history(self):
        return self.history

    def show_summaries(self):
        return self.summaries

In [9]:
def add_message(self, role, content):
        """Add a message to the conversation history"""
        self.history.append({"role": role, "content": content})
        self.run_count += 1

        # Check if periodic summarization needed
        if self.run_count % self.k == 0:
            self.summarize_history()


In [13]:

    def summarize_history(self):
        """Summarize conversation history and replace with summary"""
        prompt = [
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations concisely."},
            {"role": "user", "content": f"Summarize this conversation:\n\n{self.history}"}
        ]

        response = client.chat.completions.create(
            model=self.model,
            messages=prompt,
            max_tokens=150
        )

        summary = response.choices[0].message.content.strip()
        self.summaries.append(summary)

        self.history = [{"role": "system", "content": f"Summary so far: {summary}"}]


In [16]:
def truncate_by_turns(self, n=5):
        """Keep only the last n messages"""
        if len(self.history) > n:
            self.history = self.history[-n:]

def truncate_by_length(self, max_chars=500):
        """Keep only up to max_chars of recent conversation"""
        total = 0
        truncated = []
        for msg in reversed(self.history):
            if total + len(msg["content"]) <= max_chars:
                truncated.insert(0, msg)
                total += len(msg["content"])
            else:
                break
        self.history = truncated

def show_history(self):
        return self.history

def show_summaries(self):
        return self.summaries

In [23]:
conv = ConversationManager(k=3)
# Simulated conversation
samples = [
    ("user", "Hi, I want to learn Python."),
    ("assistant", "Great! Python is beginner-friendly. Do you know about variables?"),
    ("user", "Yes, I know a bit. What about loops?"),
    ("assistant", "Loops allow repeating actions. For example, a for loop."),
    ("user", "Can you show me an example with numbers?"),
    ("assistant", "Sure! for i in range(5): print(i) will print 0–4."),
    ("user", "Awesome, what about functions?")
]

for role, msg in samples:
    conv.add_message(role, msg)
    print(f"Added: {role}: {msg}")
    print("History now:", conv.show_history())
    print("Summaries:", conv.show_summaries())
    print("-"*50)

# -----------------------------
# Truncation Examples
# -----------------------------
print("\n🔹 Truncation by turns (last 3 messages):")
conv.truncate_by_turns(3)
print(conv.show_history())

print("\n🔹 Truncation by length (max 100 chars):")
conv.truncate_by_length(100)
print(conv.show_history())

Added: user: Hi, I want to learn Python.
History now: [{'role': 'user', 'content': 'Hi, I want to learn Python.'}]
Summaries: []
--------------------------------------------------
Added: assistant: Great! Python is beginner-friendly. Do you know about variables?
History now: [{'role': 'user', 'content': 'Hi, I want to learn Python.'}, {'role': 'assistant', 'content': 'Great! Python is beginner-friendly. Do you know about variables?'}]
Summaries: []
--------------------------------------------------
Added: user: Yes, I know a bit. What about loops?
History now: [{'role': 'system', 'content': 'Summary so far: This conversation is about learning Python:\n\n- The user wants to learn Python.\n- The assistant confirms Python is easy to learn and asks if the user knows about variables.\n- The user acknowledges knowing variables to some extent and also has a basic understanding of loops.'}]
Summaries: ['This conversation is about learning Python:\n\n- The user wants to learn Python.\n- The ass

In [26]:
import json
from jsonschema import validate, ValidationError

# Strict JSON schema
schema = {
    "type": "object",
    "required": ["name", "email", "phone", "location", "age"],
    "additionalProperties": False,
    "properties": {
        "name": {"type": "string", "minLength": 1},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string", "pattern": "^[0-9()+\\-\\s]{7,20}$"},
        "location": {"type": "string", "minLength": 1},
        "age": {"type": "integer", "minimum": 0, "maximum": 120}
    }
}

# Define the model to be used
MODEL = "llama-3.1-8b-instant"

def extract_structured_data_from_chat(chat_text: str, client=client, model=MODEL, max_retries=2):
    schema_str = json.dumps(schema, indent=2, ensure_ascii=False)
    system_instructions = (
        "You are an extractor. Given a chat message, produce a single JSON object EXACTLY matching the provided JSON Schema.\n"
        "Output ONLY the JSON object and nothing else. If a field cannot be found, set a reasonable default: empty string for strings and -1 for age (but ensure types).\n"
    )
    prompt = (
        f"{system_instructions}\nJSON Schema:\n{schema_str}\n\nChat message:\n{chat_text}\n\nReturn the JSON object now.")

    raw_response = None
    last_exc = None
    for attempt in range(max_retries):
        try:
            resp = client.chat.completions.create(model=model, messages=[{"role": "user", "content": prompt}]) # Corrected API call
            raw_response = getattr(resp, 'output_text', None) or ''
            if not raw_response:
                try:
                    raw_response = resp.choices[0].message.content # Corrected API call
                except Exception:
                    raw_response = ''
            break
        except Exception as e:
            last_exc = e

    if raw_response is None:
        raise RuntimeError(f"Failed to get response from model: {last_exc}")

    raw_text = raw_response.strip()

    # Attempt to extract JSON substring
    try:
        first = raw_text.index('{')
        last = raw_text.rindex('}') + 1
        json_text = raw_text[first:last]
        parsed = json.loads(json_text)
    except Exception:
        try:
            parsed = json.loads(raw_text)
        except Exception as e:
            return None, raw_text, f"JSON parse error: {e}"

    # Validate
    try:
        validate(instance=parsed, schema=schema)
        return parsed, raw_text, None
    except ValidationError as ve:
        return parsed, raw_text, f"ValidationError: {ve.message} at {list(ve.path)}"

# Demonstration with 3 samples
if 'client' in globals():
    sample_chats = [
        "Hi, I'm Priya Sharma. You can reach me at priya.sharma@example.com or +91 98765 43210. I live in Bangalore and I'm 27 years old.",
        "Hello — name is John Doe. Email johnny.d@example.org. Phone (415) 555-2671. Location: San Francisco. Age: 34.",
        "Name: María González\nEmail: maria.g@correo.mx\nPhone: +52 1 55 1234 5678\nLocation: Mexico City\nAge: 45"
    ]

    for i, chat in enumerate(sample_chats, start=1):
        parsed, raw, error = extract_structured_data_from_chat(chat)
        print('=== Sample', i, '===')
        print('Chat:', chat)
        print('Raw model output:', raw)
        print('Parsed:', json.dumps(parsed, indent=2, ensure_ascii=False) if parsed else None)
        print('Error:', error)
        print('\n')
else:
    print('Client not set. Set GROQ_API_KEY and re-run to demonstrate Task 2.')

=== Sample 1 ===
Chat: Hi, I'm Priya Sharma. You can reach me at priya.sharma@example.com or +91 98765 43210. I live in Bangalore and I'm 27 years old.
Raw model output: {
  "name": "Priya Sharma",
  "email": "priya.sharma@example.com",
  "phone": "+91 98765 43210",
  "location": "Bangalore",
  "age": 27
}
Parsed: {
  "name": "Priya Sharma",
  "email": "priya.sharma@example.com",
  "phone": "+91 98765 43210",
  "location": "Bangalore",
  "age": 27
}
Error: None


=== Sample 2 ===
Chat: Hello — name is John Doe. Email johnny.d@example.org. Phone (415) 555-2671. Location: San Francisco. Age: 34.
Raw model output: {
  "name": "John Doe",
  "email": "johnny.d@example.org",
  "phone": "(415) 555-2671",
  "location": "San Francisco",
  "age": 34
}
Parsed: {
  "name": "John Doe",
  "email": "johnny.d@example.org",
  "phone": "(415) 555-2671",
  "location": "San Francisco",
  "age": 34
}
Error: None


=== Sample 3 ===
Chat: Name: María González
Email: maria.g@correo.mx
Phone: +52 1 55 1234 567